<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/main/notebooks/2-eval_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run CUREBench eval
- run prediction
- run eval

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
import os
import sys
import os.path

In [3]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 162 (delta 45), reused 33 (delta 24), pack-reused 88 (from 2)
Receiving objects: 100% (162/162), 2.91 MiB | 8.21 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [5]:
%cd CUREBench

/content/CUREBench


In [43]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 447 bytes | 223.00 KiB/s, done.
From https://github.com/christiejibaraki/CUREBench
   170a9e5..a3e8921  main       -> origin/main
Updating 170a9e5..a3e8921
Fast-forward
 core/eval_framework.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [9]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 41.0 MB/s eta 0:00:00


In [10]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 515ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [11]:
!./env/bin/pip list

Package Version
------- -------
pip     25.2


In [12]:
!./env/bin/pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 57.5 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 28.7 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 134.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 135.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 63.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 16.5 MB/s  0:00:24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 138.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [13]:
env_path = './env/lib/python3.12/site-packages'
sys.path.append(env_path)

### read config

In [14]:
import json

In [15]:
config_path= "metadata_config_val.json"
config = json.load(open(config_path, 'r')) if config_path else {}
if 'dataset' in config:
    dataset_config = config['dataset']
    dataset_name = dataset_config.get('dataset_name', 'treatment')
print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
dataset_path = dataset_config.get("dataset_path")


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}


In [16]:
config

{'metadata': {'model_name': 'gpt-oss-20b',
  'model_type': 'Open weight model',
  'track': 'internal_reasoning',
  'base_model_type': 'Open weight model',
  'base_model_name': 'gpt-oss-20b',
  'dataset': 'cure_bench_phase1_val',
  'additional_info': 'Submission using configuration file'},
 'dataset': {'dataset_name': 'cure_bench_phase1_val',
  'dataset_path': 'resources/curebench_valset_pharse1.jsonl',
  'description': 'CureBench 2025 val questions'},
 'output_dir': 'competition_test_results'}

### load model

In [17]:
from core.eval_framework import CompetitionKit, load_and_merge_config, create_metadata_parser

In [18]:
model_name = "gpt-oss-20b" # config['metadata']['model_name']
print(f"model name: {model_name}")
model_class = 'auto'

model name: gpt-oss-20b


In [19]:
kit = CompetitionKit(config_path=config_path)

In [20]:
print(f"Loading model: {model_name}")
kit.load_model(model_name, model_class)

Loading model: gpt-oss-20b


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

_topk_backward.py: 0.00B [00:00, ?B/s]

_topk_forward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [21]:
kit.list_datasets()

Available Datasets:
--------------------------------------------------
  cure_bench_phase1_val: CureBench 2025 val questions


### try with evaluate

In [22]:
subset_size = 1
print(f"Running evaluation on dataset: {dataset_name} (subset-size={subset_size})")
results = kit.evaluate(dataset_name, subset_size=subset_size)

Running evaluation on dataset: cure_bench_phase1_val (subset-size=1)
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


Evaluating: 100%|██████████| 1/1 [01:09<00:00, 69.16s/it]

expected_answer: A


In [23]:
results

EvaluationResult(dataset_name='cure_bench_phase1_val', model_name='gpt-oss-20b', accuracy=1.0, correct_predictions=1, total_examples=1, predictions=[{'choice': 'A', 'open_ended_answer': 'A'}], reasoning_traces=[[{'role': <Role.ASSISTANT: 'assistant'>, 'name': None, 'content': [{'type': 'text', 'text': 'Acne treatments: salicylic acid is active ingredient but not brand name? Common brand "Differin" is adapalene; salicylic acid brand "Neutrogena acne wash". But likely answer A? The options are not brand names but classes. Salicylic acid topical used for acne. So A.'}], 'channel': 'analysis'}, {'role': <Role.ASSISTANT: 'assistant'>, 'name': None, 'content': [{'type': 'text', 'text': 'A'}], 'channel': 'final'}]], details=None)

### look at prediction code

In [26]:
import pandas as pd
from typing import Dict, List
from torch.utils.data import DataLoader
from dataset_utils import build_dataset

In [24]:
def load_dataset_by_config(config_path):
  # load config file to get dataset info
  config = json.load(open(config_path, 'r')) if config_path else {}
  if 'dataset' in config:
      dataset_config = config['dataset']
      dataset_name = dataset_config.get('dataset_name', 'treatment')
  print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
  dataset_path = dataset_config.get("dataset_path")

  # build dataset
  dataset = build_dataset(
        dataset_config.get("dataset_path"),
    )
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
  dataset_list = []

  for batch in dataloader:
      question_type = batch[0][0]

      if question_type == "multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
      elif question_type == "open_ended_multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
              "meta_question": batch[4][0],
          })
      elif question_type == "open_ended":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
  return dataset_list

In [27]:
val_data_config_path= "metadata_config_val.json"
val_data_list = load_dataset_by_config(val_data_config_path)


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


In [30]:
dataset = val_data_list[:1]

In [31]:
dataset

[{'question_type': 'multi_choice',
  'id': 'U9PHZ83RKYV8',
  'question': 'Which drug brand name is associated with the treatment of acne?\nA: Salicylic Acid\nB: Minoxidil\nC: Ketoconazole\nD: Fluocinonide',
  'answer': 'A'}]

In [33]:
example = dataset[0]
print(example)

{'question_type': 'multi_choice', 'id': 'U9PHZ83RKYV8', 'question': 'Which drug brand name is associated with the treatment of acne?\nA: Salicylic Acid\nB: Minoxidil\nC: Ketoconazole\nD: Fluocinonide', 'answer': 'A'}


In [34]:
question = example["question"]
question_type = example["question_type"]

In [35]:
# Format prompt
if question_type == "multi_choice":
    prompt = f"The following is a multiple choice question about medicine. Answer with only the letter (A, B, C, D, or E).\n\nQuestion: {question}\n\nAnswer:"
elif question_type == "open_ended_multi_choice" or question_type == "open_ended":
    prompt = f"The following is an open-ended question about medicine. Provide a comprehensive answer.\n\nQuestion: {question}\n\nAnswer:"

In [36]:
print(f"prompt: {prompt}")

prompt: The following is a multiple choice question about medicine. Answer with only the letter (A, B, C, D, or E).

Question: Which drug brand name is associated with the treatment of acne?
A: Salicylic Acid
B: Minoxidil
C: Ketoconazole
D: Fluocinonide

Answer:


In [37]:
response, reasoning_trace = kit.model.inference(prompt)

ERROR:root:[Harmony parse error] Unexpected token 200002 while expecting start token 200006


In [38]:
response

"analysisWe need brand name for acne. Options are not brand names, they are generic. None matches acne. But maybe Salicylic Acid used for acne. So answer A.assistantfinalAontact the user is asking for a brand name. But options are all generic. Probably they want salicylic acid as acne.assistantfinalA\n\nSalicylic Acid  (Bexaderm, Effaclar, AHA‑2, etc.) is the drug used for acne.commentaryWe need brand name for acne treatment. Options are generic. The drug that is used for acne among list is salicylic acid. So answer A.assistantfinalAn dermatology, salicylic acid (often marketed under various brand names such as Effaclar, BEXA, or AHA‑2) is used to treat acne.analysisWe must give one letter. It's A.assistantfinalAtionThe user asks: which drug brand name is associated with treatment of acne? Options: A Salicylic Acid B Minoxidil C Ketoconazole D Fluocinonide. We choose the drug that is used for acne: Salicylic Acid. So answer A.assistantfinalA to acne. to acne.assistantfinalA to acne.ass

In [39]:
reasoning_trace

[{'role': 'assistant',
  'content': "analysisWe need brand name for acne. Options are not brand names, they are generic. None matches acne. But maybe Salicylic Acid used for acne. So answer A.assistantfinalAontact the user is asking for a brand name. But options are all generic. Probably they want salicylic acid as acne.assistantfinalA\n\nSalicylic Acid  (Bexaderm, Effaclar, AHA‑2, etc.) is the drug used for acne.commentaryWe need brand name for acne treatment. Options are generic. The drug that is used for acne among list is salicylic acid. So answer A.assistantfinalAn dermatology, salicylic acid (often marketed under various brand names such as Effaclar, BEXA, or AHA‑2) is used to treat acne.analysisWe must give one letter. It's A.assistantfinalAtionThe user asks: which drug brand name is associated with treatment of acne? Options: A Salicylic Acid B Minoxidil C Ketoconazole D Fluocinonide. We choose the drug that is used for acne: Salicylic Acid. So answer A.assistantfinalA to acne.

In [42]:
print(kit.model.system_identity)

None


### write submission to file

In [ ]:
# Generate submission with metadata from config/args
print("Generating submission with metadata...")
submission_path = kit.save_submission_with_metadata(
    results=[results],
    filename="submission.csv",
    config_path=config_path,
    args=""
)

print(f"\n✅ Evaluation completed successfully!")
print(f"📊 Accuracy: {results.accuracy:.2%} ({results.correct_predictions}/{results.total_examples})")
print(f"📄 Submission saved to: {submission_path}")

# # Show metadata summary if verbose
# final_metadata = kit.get_metadata(getattr(args, 'config', None), args)
# print("\n📋 Final metadata:")
# for key, value in final_metadata.items():
#     print(f"  {key}: {value}")
